# All TCGA cancers analysis

In [ ]:
import os
os.environ['NUMEXPR_MAX_THREADS'] = '112'
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import torch, sys, datetime, re, optuna, umap, warnings, scipy.stats
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler, label_binarize
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, roc_auc_score, average_precision_score, balanced_accuracy_score, cohen_kappa_score, log_loss
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.pipeline import Pipeline
plt.style.use('ggplot')
# Custom functions:
from classification import *
plt.style.use('ggplot')
device = "cpu"
today = datetime.datetime.now().strftime("%Y%m%d")

In [ ]:
save_dir = os.path.join('.',
                        f'data/interim/{today}_all_cancers_classification')
os.makedirs(save_dir,exist_ok=True)
print(save_dir)

In [ ]:
def cancer_classification(histological_type_i,rnaseq,clinical_here,save_dir,
                          features=None,
                          n_threads=os.cpu_count(),
                          n_trials_optuna=100,
                          n_br=100,
                          classification_type='weighted',
                          stage_classification='i_ii_iii_iv',
                          possible_stages=['Stage I','Stage II', 'Stage III', 'Stage IV'],
                          scaler=None,
                          per=20,
                          test_size=0.2):
    '''
    Classify cancer histological type. This function follows these steps:
    1. Extract corresponding stage and RNASeq information of given cancer
    2. Preprocessing: removing lowly-expressed and lowly-variance genes.
    3. Stage classification, based on the rnaseq data of the patients with the histological type  and the stage data from the clinical dataset. 
    Classification is performed through xgboost, after optimizing the hyperparameters with optuna.
    
    :param histological_type_i: cancer type to classify
    :param rnaseq: rnaseq data for patients and genes
    :param clinical_here: clinical data, including histological type and stage
    :param save_dir: path to save the classification results
    :param features: features (genes) to select from rnaseq for classification
    :param n_threads: number of threads to use for parallel processing
    :param n_trials_optuna: number of optuna trials
    :param n_br: number of boosting rounds for xgboost
    :param classification_type: must begin with 'weighted' or 'unbalanced'
    :param stage_classification: type of classification to perform
    :param possible_stages: use only these stages for classification; can also be 'early' and 'late'
    :param scaler: scaler to use for data preprocessing or pipeline of scalers
    :param per: percentage of zeros in rnaseq to remove genes during preprocessing
    :param test_size: test size for classification
    :return: 
    '''
    assert classification_type.startswith('weighted') or classification_type.startswith('unbalanced')
    assert stage_classification in ['i_ii_iii_iv','i_ii_iii','early_late']
    assert possible_stages in [['Stage I','Stage II', 'Stage III', 'Stage IV'],['Stage I','Stage II', 'Stage III'],['early','late']]
    assert scaler is None or isinstance(scaler,StandardScaler) or isinstance(scaler,Pipeline) or isinstance(scaler,MinMaxScaler)
    assert isinstance(per,int) and per>0 and per<100
    # Set random seed
    seed_here = np.random.randint(0,1e6)
    # Save path
    print(histological_type_i)
    save_dir_i = os.path.join(save_dir,histological_type_i)
    os.makedirs(save_dir_i,exist_ok=True)
    # Get clinical data of histological type i patients
    clinical_i = clinical_here[clinical_here['histological_type']==histological_type_i]
    # Get rnaseq data of histological type i patients
    rnaseq_i = rnaseq.loc[:,rnaseq.columns.isin(clinical_i.index)]
    # Feature selection or data preprocessing
    if features is not None:
        rnaseq_i = rnaseq_i.loc[features]
        # Save rnaseq dataset:
        rnaseq_i.to_csv(os.path.join(save_dir_i,'rnaseq_features.csv'))
    # Remove lowly-expressed genes:
    else:
        # Keep genes with 0 expression in at least 20% of the patients
        rnaseq_redux = rnaseq_i[(rnaseq_i==0).sum(axis=1)/rnaseq_i.shape[1]<=per/100]
        # Keep genes whose mean expression and variance is equal or above 0.5 (in both cases).
        rnaseq_i = rnaseq_redux.iloc[(np.mean(rnaseq_redux, axis=1).values >= 0.5) &
                               (np.var(rnaseq_redux, axis=1).values >= 0.5)]
        # Save rnaseq dataset:
        rnaseq_i.to_csv(os.path.join(save_dir_i,'rnaseq_preprocessed.csv'))
    # Refine selected stages
    clinical_subset = clinical_i[clinical_i['ajcc_pathologic_tumor_stage'].isin(possible_stages)]
    num_classes = len(clinical_subset['ajcc_pathologic_tumor_stage'].unique())
    # Obtain corresponding rnaseq info
    rnaseq_subset = rnaseq_i.loc[:,np.isin(rnaseq_i.columns, clinical_subset.index)]
    # Check patients in clinical and rnaseq data coincides
    if clinical_subset.shape[0] != rnaseq_subset.shape[1]:
        clinical_subset = clinical_subset[clinical_subset.index.isin(rnaseq_subset.columns)]
    # Save clinical data
    clinical_subset.to_csv(os.path.join(save_dir_i,'clinical_subset.csv'))
    # Check for no patients
    if clinical_subset.shape[0]==0 or rnaseq_subset.shape[1]==0:
        print(f'skipping {histological_type_i} classification because no patients were found')
        return
    # Scale rnaseq data
    if scaler is not None:
        cols_rnaseq = rnaseq_subset.columns
        rows_rnaseq = rnaseq_subset.index
        rnaseq_subset = scaler.fit_transform(rnaseq_subset)
        # Save scaled rnaseq dataset:
        pd.DataFrame(rnaseq_subset,columns=cols_rnaseq,index=rows_rnaseq).to_csv(os.path.join(save_dir_i,'rnaseq_scaled.csv'))
    # Make classification
    classification_type_fun = 'weighted' if classification_type.startswith('weighted') else 'unbalanced'
    unbalanced_classification = classification_benchmark(
        X_data=rnaseq_subset.T,
        y_data=clinical_subset['ajcc_pathologic_tumor_stage'],
        classification_type = classification_type_fun,
        num_classes=num_classes,
        seed=seed_here, 
        test_size=test_size,
        n_br=n_br,
        num_threads = n_threads,
        n_trials = n_trials_optuna,
    )
    
    return unbalanced_classification, save_dir_i, seed_here, classification_type_fun

## Load data

In [ ]:
# RNA-seq data:
rnaseq = pd.read_table("data/raw/BatchEffectsNormalizedmRNAdata_EB++AdjustPANCAN_IlluminaHiSeq_RNASeqV2.geneExp.xena", 
                       index_col=0, 
                       sep="\t")
print(rnaseq.shape)

In [ ]:
# Remove duplicated gene
rnaseq = rnaseq.loc[~rnaseq.index.duplicated(),:]
rnaseq.shape

In [ ]:
# Sample type and primary disease data:
pheno = pd.read_table("data/raw/SampleTypeAndPrimaryDisease_TCGA_phenotype_denseDataOnlyDownload.tsv",sep="\t",index_col=0)
print(pheno.shape)
print(pheno.columns)

In [ ]:
pheno_primary=pheno[pheno['sample_type']=='Primary Tumor']
print(pheno_primary.shape)

In [ ]:
# Load the data
clinical_op = pd.read_csv('data/raw/CuratedClinicalData_Survival_SupplementalTable_S1_20171025_xena_sp', sep='\t', index_col=0)
clinical=clinical_op[['cancer type abbreviation', 'ajcc_pathologic_tumor_stage', 'histological_type']]
clinical = clinical.dropna(subset=['ajcc_pathologic_tumor_stage'])
print(clinical.shape)
print(clinical.columns)

## Select stages

In [ ]:
# Merge sample index pheno_primary and clinical
clinical_merge=clinical[clinical.index.isin(pheno_primary.index)]
print(clinical_merge.shape)

In [ ]:
# Remove stage cases with: [Discrepancy], IS, Stage X, [Unknown], I/II NOS
clinical_merge=clinical_merge[
    ~clinical_merge['ajcc_pathologic_tumor_stage'].isin(
        ['[Discrepancy]','IS','Stage X','[Unknown]','I/II NOS'])
]
print(clinical_merge.shape)

In [ ]:
# Remove A, B or C on each substage:
stages_clump = []
for i in clinical_merge["ajcc_pathologic_tumor_stage"]:
    if i.endswith("A") or i.endswith("B") or i.endswith("C"):
        stages_clump.append(i[:-1])
        #print(i[:-1])
    else:
        stages_clump.append(i)
        
len(stages_clump)

# Assign new stages:
clinical_merge["ajcc_pathologic_tumor_stage"] = stages_clump

In [ ]:
# In clinical_merge, keep only the 10 most common cancer type abbreviations
keep_cancer_types = clinical_merge['cancer type abbreviation'].value_counts().index[:10]
clinical_merge = clinical_merge[clinical_merge['cancer type abbreviation'].isin(keep_cancer_types)]

In [ ]:
# In clinical_merge, keep only the 10 most common histological types
keep_histological_types = clinical_merge['histological_type'].value_counts().index[:10]
clinical_merge = clinical_merge[clinical_merge['histological_type'].isin(keep_histological_types)]
clinical_merge[['cancer type abbreviation','histological_type']].value_counts()

In [ ]:
# assigning early/late labels to I,II/III,IV, respectively:
dict_labels = {'Stage I':'early','Stage II':'early', 'Stage III':'late', 'Stage IV':'late'}
clinical_early_late = clinical_merge.replace(dict_labels)

---

## Classification pipeline

In the following cell, all possible parameters for the `cancer_classification` function are stated and explained.

Comment out the different parameters to choose the behavior of `cancer_classification`.

In [ ]:
#################
# Parameters
#################
# Number of threads
n_threads = 16
# Number of optuna trials to run to find optimal hyperparameters
n_trials_optuna = 100
# Number of boosting rounds for xgboost
n_br = 100

# Clinical information: clinical_merge contains the specific stages (I to IV), 
# while clinical_early_late contains the early/late division of stages.
# clinical_here = clinical_merge
clinical_here = clinical_early_late

# Classification type: must start with 'weighted' or 'unbalanced'
# use an underscore _ and another word to further describe classification (eg unscaled if no scaling is used before classification)
# This is the name of the path to save the classification results to.
classification_type = 'weighted_unscaled'

# Type of stage classification.
# May be stages from I to III, I to IV, or early vs late (I and II vs III and IV)
# stage_classification = 'i_ii_iii_iv'
# stage_classification = 'i_ii_iii'
stage_classification = 'early_late'

# List of possible stages to classify:
# possible_stages = ['Stage I','Stage II', 'Stage III', 'Stage IV']
# possible_stages = ['Stage I','Stage II', 'Stage III']
possible_stages = ['early','late']

# Scaling steps for RNASeq before classification:
#scaler = Pipeline(steps=[('scaler1', StandardScaler()),('scaler2', MinMaxScaler())])
scaler = None

# when detected this percentage of zeros in rnaseq, remove corresponding genes
per = 20

# If needed, a list of genes (features) may be selected to run the classification on
features = None

In [ ]:
#################
# Classification
#################
classification_results = []
rnaseq_cancers = []
# Run through given cancer types (histological types):
for histological_type_i in keep_histological_types:
    # Run classification pipeline on selected cancer
    classification_results_i, save_dir_i, seed_here, classification_type_fun = cancer_classification(histological_type_i=histological_type_i,
                          rnaseq=rnaseq,
                          clinical_here=clinical_here,
                          save_dir=save_dir,
                          features=features,
                          n_threads=n_threads,
                          n_trials_optuna=n_trials_optuna,
                          n_br=n_br,
                          classification_type=classification_type,
                          stage_classification=stage_classification,
                          possible_stages=possible_stages,
                          scaler=scaler,
                          per=per,
                          test_size=0.2)
    # save classification results
    classification_results.append(classification_results_i)
    #rnaseq_cancers.append(rnaseq_i)
    # plot results
    (_, metrics_unbalanced, _, _, _, _) = classification_results_i
    fig, ax = plt.subplots(figsize=(24, 7))
    plot_metrics(metrics_here=metrics_unbalanced, 
                 title=f'{classification_type_fun}. \nCancer histological type: {histological_type_i}',ax=ax,
                 save_path=None,
                 save_name='metrics_unbalanced',
                 show_legend=True,show_plot=False)
    save_classification_i = os.path.join(save_dir_i,classification_type,stage_classification)
    os.makedirs(save_classification_i,exist_ok=True)
    # plt.savefig(os.path.join(save_classification_i,'classification_results.png'),
    #             bbox_inches='tight',
    #             dpi=300,
    #             format='png')
    # plt.savefig(os.path.join(save_classification_i,'classification_results.pdf'),
    #             bbox_inches='tight',
    #             format='pdf')
    plt.savefig(os.path.join(save_classification_i,'classification_results.svg'),
                bbox_inches='tight',
                format='svg')
    plt.show()
    print()
    